# Liver Model Construction: Huge Core Model

In [1]:
import os
import re
import warnings
from cobra.io import *
from cobra.io.json import *
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa, qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile, plot_ensemble_phase_portrait, plot_ensemble_time_profile
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
# adding Folder_2 to the system path
sys.path.insert(0, "c:\\Users\\sicil\\LiverModel")
from cobra_dict import *
import cobra_dict as c_d
import csv
import altair as alt
from minspan.minspan import minspan,nnz
import sys
# from minspan import minspan,nnz

MASSpy version: 0.1.6


In [3]:
mass.__version__

'0.1.6'

In [2]:
maps_dir = os.path.abspath("maps")
data_dir = os.path.abspath("data")
dir = os.path.abspath("")
model_dir = os.path.abspath("models")
minspan_dir= os.path.abspath("minspans_csv")


### Getting reactions and metabolite data from RECON 3D

In [3]:
# making dataframe of metabolites

csv_met = os.path.join(data_dir,"big_core_met_df.csv")
met_csv_df = pd.read_csv(csv_met,index_col="id")
metabolite_info=met_csv_df.to_dict(orient='index')



# making dataframe of reactions
csv_rxn = os.path.join(data_dir,"big_core_reaction_df.csv")
rxn_csv_df = pd.read_csv(csv_rxn,index_col="id") 
reaction_info = rxn_csv_df.to_dict(orient='index')

### Loading cobra model

In [30]:
# Initiate empty model
trial= cobra.Model()

In [31]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    trial.add_metabolites(met)

In [32]:
# Loading reaction data as JSON file to maintain data types as dictionaries 
rxn_json = os.path.join(data_dir,"big_core_reaction_df.json")
with open(rxn_json) as json_file:
    rxn_data = json.load(json_file)

In [33]:
for reaction, info in rxn_data.items():
    reaction_obj = cobra.Reaction(reaction)
    reaction_obj.id=reaction
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    trial.add_reaction(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)
    # print(reaction)

In [34]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

CSm {'charge': -1.0}
ACONTm {}
ICDHxm {}
AKGDm {}
SUCOASm {'charge': 2.0}
SUCD1m {}
FUMm {}
MDHm {'charge': -1.0}
NADH2_u10mi {'charge': 1.0}
FADH2ETC {}
CYOR_u10mi {'charge': -2.0}
CYOOm2i {'charge': 4.0}
ATPS4mi {'charge': -1.0}
Htmi {}
HEX1 {'charge': -1.0}
PGI {}
PFK {'charge': -1.0}
FBA {}
TPI {}
GAPD {'charge': -3.0}
PGK {}
PGM {}
ENO {}
PYK {'charge': 1.0}
ADK1 {}
PDHm {}
LDH_L {'charge': -1.0}
G6PDH2r {'charge': -1.0}
PGL {'charge': -1.0}
GND {}
RPI {}
RPE {}
TKT1 {}
TKT2 {}
TALA {}
PCm {'charge': 1.0}
PEPCKm {}
FBP {'charge': 2.0}
G6PPer {'charge': 2.0}
G6Pter {}
GLCter {}
MALtm {}
MDH {'charge': -1.0}
PEPCK_re {}
PGMT {}
GALUi {'charge': 1.0}
GLGNS1 {'charge': -3.0}
GLBRAN {}
GLPASE1 {'charge': -6.0}
GLDBRAN {}
GLPASE2 {}
GGNG {'charge': -8.0}
ACACT1m {}
HMGCOASm {'charge': 1.0}
HMGLm {}
ADCim {'charge': 1.0}
BDHm {'charge': -1.0}
OCOAT1m {}
NDPK1 {}
NDPK1m {}
NDPK2 {}
HCO3Em {'charge': -1.0}
ATPtm {}
AKGMALtm {}
ASPTA {}
ASPTAm {}
CITtam {}
CITL {}
ME2 {}
ACS {}
ARGN {}
OCBT

In [35]:
trial.metabolites.h_c.charge= 1
trial.metabolites.h_m.charge= 1
trial.metabolites.h_i.charge= 1
trial.metabolites.pi_c.charge= -2
trial.metabolites.pi_m.charge= -2

In [36]:
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

CSm {}
ACONTm {}
ICDHxm {}
AKGDm {}
SUCOASm {}
SUCD1m {}
FUMm {}
MDHm {}
NADH2_u10mi {}
FADH2ETC {}
CYOR_u10mi {}
CYOOm2i {}
ATPS4mi {}
Htmi {}
HEX1 {}
PGI {}
PFK {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}
ADK1 {}
PDHm {}
LDH_L {}
G6PDH2r {}
PGL {}
GND {}
RPI {}
RPE {}
TKT1 {}
TKT2 {}
TALA {}
PCm {}
PEPCKm {}
FBP {}
G6PPer {'charge': 2.0}
G6Pter {}
GLCter {}
MALtm {}
MDH {}
PEPCK_re {}
PGMT {}
GALUi {}
GLGNS1 {}
GLBRAN {}
GLPASE1 {}
GLDBRAN {}
GLPASE2 {}
GGNG {}
ACACT1m {}
HMGCOASm {}
HMGLm {}
ADCim {}
BDHm {}
OCOAT1m {}
NDPK1 {}
NDPK1m {}
NDPK2 {}
HCO3Em {}
ATPtm {}
AKGMALtm {}
ASPTA {}
ASPTAm {}
CITtam {}
CITL {}
ME2 {}
ACS {}
ARGN {}
OCBTm {}
ARGSS {}
ARGSL {}
CBPSam {}
NH4tm {}
GDHm {}
GLNS {}
GLUN {}
ALATA_L {}
CYSO {}
3SALATAi {}
3SPYRSP {}
PGCD {}
PSERT {}
PSP_L {}
GHMT2r {}
ASNS1 {}
GLYtm {}
MLTHFtm {}
THFtm {}
KHK {}
FBA2 {}
TRIOK {}
ALCD19y {}
GLYK {}
2AMACHYD {}
SERHL {}
ALCD2x {}
ACALDtm {}
ALDD2xm {}
ACSm {}
Htm {}
O2tm {}
H2Otr {}
H2Otm {}
PItm {}
PItr {'charge'

In [63]:
trial.objective = 'ATPS4mi'
flux= trial.optimize()
df= flux.to_frame()
df.loc["EX_glc__D_c"]
# flux.loc["EX_glc__D_c"]

fluxes           -1.0
reduced_costs   -56.0
Name: EX_glc__D_c, dtype: float64

In [65]:
for r in trial.reactions:
    if 'EX_' in r.id:
        print(r.id, r.lower_bound, r.upper_bound)

EX_h_c -1000.0 1000.0
EX_pyr_c 0.0 1000.0
EX_h2o_c -1000.0 1000.0
EX_pi_c -1000.0 1000.0
EX_glc__D_c -1.0 1000.0
EX_lac__L_c 0.0 1000.0
EX_co2_c 0.0 1000.0
EX_o2_c -1000.0 1000.0
EX_octa_prod_c 0.0 1000.0
EX_octa_cons_c 0.0 0.0
EX_urea_c 0.0 1000.0
EX_gln__L_c 0.0 1000.0
EX_acetone_c 0.0 1000.0
EX_bhb_c 0.0 1000.0
EX_glu__L_c 0.0 1000.0
EX_ser__L_c 0.0 1000.0
EX_cys__L_c 0.0 1000.0
EX_gly_c 0.0 1000.0
EX_glygn2_c 0.0 1000.0
EX_Tyr_ggn_c -1000.0 1000.0
EX_ala__L_c 0.0 1000.0
EX_nh4_c -1000.0 0.0
EX_so3_c 0.0 1000.0
EX_etoh_c 0.0 1000.0
EX_glyc_3octa_c 0.0 1000.0
EX_fru_c 0.0 1000.0


In [ ]:
#Function to load data from the excel sheet
# def load_data(filepath, sheet_name):
#     """Load Liver data from an excel sheet"""
#     df = pd.read_excel(engine='openpyxl',
#         io=filepath,
#         sheet_name=sheet_name,
#         index_col=0)
#     return df

In [66]:
#define data sheet as latest data sheet
data_sheet=os.path.join(data_dir,"fluxes.csv")

flux_df = pd.read_csv(
    data_sheet)
flux_df = flux_df.dropna() #drops rows with NaN
flux_df= flux_df.set_index("External Metabolite")
# flux_df.index
flux_df

,fasted_upper,fasted_lower,fed_upper,fed_lower
External Metabolite,,,,
glc__D_c,37.18,28.10,62.910,48.670
urea_c,53.83,49.69,63.090,57.390
nh4_c,5.82,-3.28,-22.430,-25.890
asp__L_c,0.81,0.25,1.040,0.480
glu__L_m,6.92,6.10,10.960,9.400
ser__L_c,-3.20,-4.90,-2.480,-4.520
asn__L_c,0.17,-0.13,-0.070,-0.090
gly_c,-2.48,-3.54,-3.110,-4.930
gln__L_c,-31.17,-41.97,-28.200,-33.040


In [67]:
for i in flux_df.index:
    # print(i)
    for r in trial.reactions:
        if str(i) in r.id:
    # if i in trial.reactions:
            print(r.id)
            r.lower_bound = flux_df.loc[i,"fasted_lower"]
            r.upper_bound = flux_df.loc[i,"fasted_upper"]
    # print(trial.reactions.__contains__(i))

EX_glc__D_c
EX_urea_c
EX_nh4_c
EX_ser__L_c
EX_gly_c
EX_gln__L_c
EX_ala__L_c
EX_cys__L_c


In [21]:
for r in trial.reactions:
    if 'EX_' in r.id:
        if r.lower_bound == 0:
            r.lower_bound= -1000
        if r.upper_bound ==0:
            r.upper_bound = 1000
        print(r.id, r.lower_bound, r.upper_bound)

EX_h_c -1000.0 1000.0
EX_pyr_c -1000 1000.0
EX_h2o_c -1000.0 1000.0
EX_pi_c -1000.0 1000.0
EX_glc__D_c 28.1 37.18
EX_lac__L_c -1000 1000.0
EX_co2_c -1000 1000.0
EX_o2_c -1000.0 1000.0
EX_octa_prod_c -1000 1000.0
EX_octa_cons_c -1000 1000
EX_urea_c 49.69 53.83
EX_gln__L_c -41.97 -31.17
EX_acetone_c -1000 1000.0
EX_bhb_c -1000 1000.0
EX_glu__L_c -1000 1000.0
EX_ser__L_c -4.9 -3.2
EX_cys__L_c 0.07 0.45
EX_gly_c -3.54 -2.48
EX_glygn2_c -1000 1000.0
EX_Tyr_ggn_c -1000.0 1000.0
EX_ala__L_c 0.8 1.36
EX_nh4_c -3.28 5.82
EX_so3_c -1000 1000.0
EX_etoh_c -1000 1000.0
EX_glyc_3octa_c -1000 1000.0
EX_fru_c -1000 1000.0


In [71]:
trial.reactions.EX_glc__D_c.lower_bound = -28.1

In [73]:
trial.objective = 'ATPS4mi'
flux_solution = trial.optimize()
flux_solution

c:\Users\sicil\miniconda3\envs\massliver\lib\site-packages\cobra\util\solver.py:554 UserWarning: Solver status is 'infeasible'.


<Solution infeasible at 0x17921cda6d0>